<a href="https://colab.research.google.com/github/Astuary/AspectBasedSentimentAnalysis/blob/main/BERT_DK_IMDB_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
for dirname, _, filenames in os.walk('drive/My Drive/Colab Notebooks'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

drive/My Drive/Colab Notebooks/train.csv
drive/My Drive/Colab Notebooks/test.csv
drive/My Drive/Colab Notebooks/Copy of CS685_HW0.ipynb
drive/My Drive/Colab Notebooks/CS685_HW0_Kunjal_Panchal.ipynb
drive/My Drive/Colab Notebooks/CS685 Quiz 1: Neural LMs.gdoc
drive/My Drive/Colab Notebooks/CS685 Quiz 2: Attention.gdoc
drive/My Drive/Colab Notebooks/CS685 Quiz 3: Transformers   transfer.gdoc
drive/My Drive/Colab Notebooks/MLFL (24th September) - John Wieting (CMU   Google Research).gdoc
drive/My Drive/Colab Notebooks/CS685 Quiz 4: BERT MLM.gdoc
drive/My Drive/Colab Notebooks/CS685 Quiz 5: text generation.gdoc
drive/My Drive/Colab Notebooks/CS685_HW1.ipynb
drive/My Drive/Colab Notebooks/data_distribute.npz
drive/My Drive/Colab Notebooks/BERT-PT-REST-TRIAL.ipynb
drive/My Drive/Colab Notebooks/NN.ipynb
drive/My Drive/Colab Notebooks/IMDB Dataset.csv.zip
drive/My Drive/Colab Notebooks/IMDB Dataset.csv
drive/My Drive/Colab Notebooks/BERT_XD_REVIEW_IMDB.ipynb
drive/My Drive/Colab Notebooks/BER

In [2]:
import sys
import numpy as np
import random as rn
import pandas as pd
import torch
from torch import nn
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

! pip install transformers
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, AdamW

In [3]:
rn.seed(1)
np.random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed(1)

In [4]:
path = 'drive/My Drive/Colab Notebooks/'
train_data = pd.read_csv(path + 'train.csv')
test_data = pd.read_csv(path + 'test.csv')

In [5]:
train_data = train_data#[:2000]
test_data = test_data#[:500]

train_data = train_data.to_dict(orient='records')
test_data = test_data.to_dict(orient='records')
type(train_data)

list

In [6]:
train_texts, train_labels = list(zip(*map(lambda d: (d['text'], d['sentiment']), train_data)))
test_texts, test_labels = list(zip(*map(lambda d: (d['text'], d['sentiment']), test_data)))

len(train_texts), len(train_labels), len(test_texts), len(test_labels)
train_texts[0]

"Now, I won't deny that when I purchased this off eBay, I had high expectations. This was an incredible out-of-print work from the master of comedy that I so enjoy. However, I was soon to be disappointed. Apologies to those who enjoyed it, but I just found the Compleat Al to be very difficult to watch. I got a few smiles, sure, but the majority of the funny came from the music videos (which I've got on DVD) and the rest was basically filler. You could tell that this was not Al's greatest video achievement (that honor goes to UHF). Honestly, I doubt if this will ever make the jump to DVD, so if you're an ultra-hardcore Al fan and just HAVE to own everything, buy the tape off eBay. Just don't pay too much for it."

In [7]:
tokenizer = AutoTokenizer.from_pretrained('activebus/BERT-DK_rest')

In [8]:
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], train_texts))
test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], test_texts))

len(train_tokens), len(test_tokens)

(25000, 25000)

In [9]:
train_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, train_tokens)), maxlen=512, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, test_tokens)), maxlen=512, truncating="post", padding="post", dtype="int")

train_tokens_ids.shape, test_tokens_ids.shape

((25000, 512), (25000, 512))

In [10]:
train_y = np.array(train_labels) == 'pos'
test_y = np.array(test_labels) == 'pos'
train_y.shape, test_y.shape, np.mean(train_y), np.mean(test_y)

((25000,), (25000,), 0.5, 0.5)

In [11]:
train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]

In [12]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()

        self.bert = AutoModel.from_pretrained('activebus/BERT-DK_rest')

        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

bert_clf = BertBinaryClassifier()
bert_clf = bert_clf.cuda() 

str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'439.069184M'

In [20]:
BATCH_SIZE = 4
EPOCHS = 10

In [21]:
train_tokens_tensor = torch.tensor(train_tokens_ids)
train_y_tensor = torch.tensor(train_y.reshape(-1, 1)).float()

test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()

train_masks_tensor = torch.tensor(train_masks)
test_masks_tensor = torch.tensor(test_masks)

str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'1765.728256M'

In [22]:
train_dataset = TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

In [17]:
param_optimizer = list(bert_clf.sigmoid.named_parameters()) 
optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

In [18]:
optimizer = Adam(bert_clf.parameters(), lr=3e-6)

torch.cuda.empty_cache()

In [23]:
for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)
        print(str(torch.cuda.memory_allocated(device)/1000000 ) + 'M')
        logits = bert_clf(token_ids, masks=masks)
        
        loss_func = nn.BCELoss()

        batch_loss = loss_func(logits, labels)
        train_loss += batch_loss.item()
        
        
        bert_clf.zero_grad()
        batch_loss.backward()
        

        clip_grad_norm_(parameters=bert_clf.parameters(), max_norm=1.0)
        optimizer.step()
        
        clear_output(wait=True)
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_data) / BATCH_SIZE, train_loss / (step_num + 1)))


Epoch:  10
6249/6250.0 loss: 0.022619232236823373 


In [25]:
bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):

        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)

        logits = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        loss = loss_func(logits, labels)
        numpy_logits = logits.cpu().detach().numpy()
        
        bert_predicted += list(numpy_logits[:, 0] > 0.5)
        all_logits += list(numpy_logits[:, 0])

In [26]:
from sklearn.metrics import classification_report
print(classification_report(test_y, bert_predicted))

              precision    recall  f1-score   support

       False       0.95      0.93      0.94     12500
        True       0.94      0.95      0.94     12500

    accuracy                           0.94     25000
   macro avg       0.94      0.94      0.94     25000
weighted avg       0.94      0.94      0.94     25000



In [35]:
torch.save(bert_clf.state_dict(), "drive/My Drive/Colab Notebooks/BERT_DK_REVIEW.pth")
tokenizer.save_pretrained("drive/My Drive/Colab Notebooks/BERT_DK_REVIEW_TKN")

('drive/My Drive/Colab Notebooks/BERT_DK_REVIEW_TKN/tokenizer_config.json',
 'drive/My Drive/Colab Notebooks/BERT_DK_REVIEW_TKN/special_tokens_map.json',
 'drive/My Drive/Colab Notebooks/BERT_DK_REVIEW_TKN/vocab.txt',
 'drive/My Drive/Colab Notebooks/BERT_DK_REVIEW_TKN/added_tokens.json')